# Aula 2 - SVM

Na aula de hoje, vamos explorar os seguintes tópicos em Python:

- 1) Introdução
- 2) Classificadores de margem
- 3) Support Vector Machines
- 4) Funções de kernel
- 5) SVM na prática com o sklearn
- 6) SVM para regressão 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ml_utils import *

____
____
____

## 1) Introdução

Na aula de hoje, falaremos sobre um dos mais interessantes métodos de aprendizagm supervisionada: **SVM** (**S**upport **V**ector **M**achines).

Este método tem uma construção extremamente elegante e robusta, que, apesar de complexa, pode ser entendida em termos geométricos simples. Nesta aula, vamos explorar os principais aspectos desta construção, em uma apresentação mais alto-nível, sem nos preocuparmos demais com os detalhes matemáticos.

Um ponto importantíssimo sobre SVMs, que é o que de fato lhes confere poder tão elevado, é sua **capacidade de produzir uma hipótese simples, a partir de um conjunto de hipóteses complexo**, o que tem consequências diretas na **capacidade de generalização** de modelos SVM: eles são capazes de generalizar muito bem, apesar das hipóteses produzidas parecerem altamente complexas, o que é algo formidável!

Não entraremos nos detalhes matemáticos que justificam e provam os pontos acima. Mas, para quem tiver interesse, quase todo livro-texto de machine learning aborda estas questões. Recomendo, em particular, [este livro](https://www.google.com.br/books/edition/Learning_with_Kernels/7r34DwAAQBAJ?hl=pt-BR&gbpv=1&dq=learning+with+kernels&printsec=frontcover), que detalha extensivamente SVM e métodos relacionados; ou então [este livro](https://cs.nyu.edu/~mohri/mlbook/), que aborda formalmente a teoria de aprendizagem, eventualmente culminando no enorme sucesso de SVMs. Por fim, aproveito também pra recomendar [este curso](https://work.caltech.edu/index.html) do Caltech, onde os tópicos abordados no livro do Mohri são apresentados de maneira clara, direta, e muito ilustrativa. Esta é minha maior recomendação para quem tem interesse em iniciar os estudos em teoria de aprendizagem estatística, e então ter todo o fundamento rigoroso pra entender porque SVMs são tão interessantes!

Agora, vamos iniciar nossa exposição, introduzindo um conceito fundamental para SVMs: a **margem**, e os chamados **classificadores de margem**.

____
____
____

## 2) Classificadores de margem

Considere o seguinte dataset supervisionado de um problema de classificação binário:

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/d8d83e07-66d8-47c6-a5e4-4a3e232481e2.PNG width=400>

É visível que os dados são linearmente separáveis. De fato, existem infinitas retas possíveis que separam perfeitamente as duas classes. Alguns exemplos:

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/bbf3d0a6-4707-471b-9f19-7290c13e4f23.PNG width=400>

Vamos olhar separadamente para cada um destes três modelos:

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/6c0bea51-dda9-400c-bb68-d7396e4fa606.PNG width=900>

Apesar de ambos separarem perfeitamente os dados de treino (erro de treino é nulo!), podemos nos perguntar: qual deles tem potencial de apresentar **melhor generalização?**

Para refletirmos sobre isso, considere que queremos classificar o ponto de teste indicado em preto:

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/fbed2deb-bb38-4ef5-8558-99b7aff5c7b7.PNG width=900>

É super razoável que este ponto seja classificado como sendo da classe vermelha, não é mesmo? No entanto, o primeiro classificador irá classificá-lo como pertencente à classe azul.

Podemos nos perguntar: por que isso acontece?

Intuitivamente, é possível dizer que a fronteira de decisão do primeiro modelo está "**muito próxima**" dos pontos da classe vermelha, não é? 

Mas, se estamos avaliando a **fronteira de decisão**, é razoável que nos importemos, na realidade, com a proximidade entre ela **e os "pontos mais externos"** da respectiva classe, não é mesmo? Afinal, intuitivamente, estes são os pontos que exercem maior influência sobre a fornteira de decisão, justo?

É aqui que entra o conceito de **margem**:

> Chamamos de **margem** a **menor distância** entre os pontos de treino e a fronteira de decisão

A seguir, visualizamos as margens associadas a cada um dos três modelos:

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/72f191fc-7bc0-4451-923a-ecb939e88111.PNG width=900>

Dada a definição de margem, fica claro que o terceiro modelo seria nossa melhor opção, pois ele **apresenta maior margem**. De fato, intuitivamente, um classificador que apresenta maior margem, terá melhores chances de generalização!

> Chamamos de **classificador de margem máxima** um classificador **linear** que é construído de modo que a margem seja maximizada.

<div class="warning" style='padding:0.1em; background-color:#E9D8FD; color:#69337A'>
<span>
<h2>Conclusões</h2>
<ul>
<li>Um classificador de margem máxima tem alto potencial de generalização, para dados linearmente separáveis;
</ul>
</span>
<br>
</div>

### Classificador de margem suave

Considere o dataset a seguir:

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/7be60cd3-e7f8-46bd-85bc-73ec1d4b0bb5.PNG width=400>

Neste caso, nós não temos mais separabilidade linear, de modo que a construção de um classificador de margem máxima não é mais possível.

Como resolver este problema?

Com a introdução do **classificador de margem suave (*soft margin classifier*)**!

Um classificador de margem suave é obtido com uma modificação extremamente simples com relação ao classificador de margem máxima: a permissão de que **erros de classificação** sejam cometidos na base de treino! Algo assim:

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/cfacd48d-9713-4bec-9fd7-ea1b4ae380f5.PNG width=400>


Neste caso, temos duas observações incorretas dentro da margem - e está tudo bem, pois dados que estas observações são outliers, não precisamos mos preocupar em ajustar a margem a elas!

Com isso, temos um modelo com um viés um pouco maior, mas com variância bem menor - caminhamos na direção do **sweet spot** de generalização!!

Note, portanto, que a introdução de uma margem suave pode ser visto como **um procedimento de regularização** aplicado ao classificador de margem! E é exatamente assim que veremos este procedimento quando chegarmos em SVM - um procedimento de regularização!

Um classificador de margem suave é também chamado de **classificador de vetores de suporte (*support vector classifier*)**, sendo que os pontos que "apoiam" a margem são justamente os chamados **vetores de suporte (*support vectors*)**.

<div class="warning" style='padding:0.1em; background-color:#E9D8FD; color:#69337A'>
<span>
<h2>Conclusões</h2>
<ul>
<li>Um classificador de margem máxima tem alto potencial de generalização, para dados linearmente separáveis;<br><br>
<li>Um classificador de margem suave (vetores de suporte) introduz um pouco mais de viés ao permitir erros de treino, mas, com isso, aumenta o potencial de generalização, e pode ser aplicável a dados que não são exatamente linearmente separáveis;
</ul>
</span>
<br>
</div>

Estamos chegando perto das SVMs!

Mas, antes de chegarmos lá, é importante frisarmos um ponto fundamental:

> A **fronteira de decisão** de um classificador de margem suave (de vetores de suporte) é **linear no espaço de features** em que o classificador é treinado, ou seja, a hipótese treinada (isto é, a superfície de decisão) será **um hiperplano** de dimensão $D-1$, onde $D$ é a dimensão do espaço de features

Vamos entender melhor o comentário acima com alguns exemplos:

### $D=2$

Se temos uma duas features$(X_1, X_2)$, teremos um **hiperplano $1-$dimensional**, que nada mais é que **uma reta** como fronteira de decisão:

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/6ca9b6d4-90fb-4d92-8a73-42ecec09d562.PNG width=400>

### $D=3$

Se temos uma três features$(X_1, X_2, X_3)$, teremos um **hiperplano $2-$dimensional**, que nada mais é que **um plano** como fronteira de decisão:

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/b91951b3-9256-4a38-abd6-287811332db8.PNG width=400>

### $D=1$

Se temos uma única feature $(X)$, teremos um **hiperplano $0-$dimensional**, que nada mais é que **um ponto** como fronteira de decisão:

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/000d5eb5-25e7-485b-bf36-6ad944970e27.PNG width=400>

Para dimensões maiores, $D > 3$, não conseguimos mais visualizar a fronteira de decisão, mas, se qualquer forma, ela será linear (por isso, um hiper**plano**!).

Agora estamos prontos para introduzir as tão aguardadas **Support Vector Machines!**

<div class="warning" style='padding:0.1em; background-color:#E9D8FD; color:#69337A'>
<span>
<h2>Conclusões</h2>
<ul>
<li>Um classificador de margem máxima tem alto potencial de generalização, para dados linearmente separáveis;<br><br>
<li>Um classificador de margem suave (vetores de suporte) introduz um pouco mais de viés ao permitir erros de treino, mas, com isso, aumenta o potencial de generalização, e pode ser aplicável a dados que não são exatamente linearmente separáveis;<br><br>
<li>A fronteira de decisão de um classificador de margem sempre será linear no espaço de features;
</ul>
</span>
<br>
</div>

____
____
____

## 3) Support Vector Machines

Imagine que temos o seguinte dataset (com uma única dimensão):

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/e00c77e0-14fb-472f-b9ae-f6af7b229ad5.PNG width=400>

É evidente que este dataset **não é linearmente separável**! Portanto, não conseguimos produzir um classificador de vetores de suporte para este dataset **no espaço de features original**.

Mas, aí entra uma ideia muito interessante: e se nós **levarmos os dados para um ou outro espaço?**

Seria possível que no espaço original os dados não sejam linearmente separáveis, mas **o sejam** em algum outro espaço?

Bom, a priori, vamos tentar algo bem simples... Que tal introduzirmos **uma nova feature** $X_2 = X_1^2$? O que aconteceria neste caso?

De fato, ao **introduzirmos** uma nova feature, estamos fazendo com que **cada observação passe a ser caracterizada por duas features ao invés de uma única**!

Ou seja, nosso espaço de features efetivamente muda de $\mathbb{R}^1$ para $\mathbb{R}^2$! Veja:

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/8d5e6199-8a33-45cf-9e4f-c6507024fb36.PNG width=800>

O procedimento que fizemos é chamado de **feature map**, e ele é matematicamente representado pelo mapa (função) $\Phi$. 

> Como $\Phi$ leva observações do espaço original ($1$D, uma única feature $X_1$, para vetores do novo espaço ($2$D, duas features, $X_1$ e $X_2$), o denotamos como: 
$$\begin{align*}
\Phi \ \colon \ & \mathbb{R}^1 \longrightarrow \mathbb{R}^2 \\
& \vec{x} = X_1 \longmapsto \Phi(\vec{x}) = (X_1, X_2)
\end{align*}$$

Note que no caso ilustrado acima, temos $X_2 = X_1^2$, isto é, $\Phi(\vec{x}) = (X_1, X_1^2)$.

Então, apesar do nome assustador, o feature map é algo que já estávamos acostumados a fazer, quando usamos o [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html), lembra? :)

Um pouco mais de terminologia:

> O "espaço original" é comumente chamado de **espaço de input** (representaremos por $\mathcal{X}$); enquanto o espaço após a aplicação do feature map é chamado de **espaço de features (representaremos por $\mathcal{Z}$)**

<img src=https://miro.medium.com/max/872/1*zWzeMGyCc7KvGD9X8lwlnQ.png width=400>

Também é comum se referir ao espaço de features como **espaço z**, devido à comum notação $\Phi(\vec{x}) \equiv \vec{z}$. Neste caso, teríamos:

$$\begin{align*}
\Phi \ \colon \ & \mathbb{R}^1 \longrightarrow \mathbb{R}^2 \\
& \vec{x} = X_1 \longmapsto \Phi(\vec{x}) = \vec{z} = (Z_1, Z_2)
\end{align*}$$

Uma vez que temos estas definições, podemos perceber a real utilidade do feature map: **os dados não eram linearmente separáveis no espaço de input, mas passaram a ser no espaço de features!**

Isso é realmente formidável, pois, se temos dados linearmente separáveis, podemos **treinar um classificador de margem suave** no espaço de features!

Isso pode parecer estranho, pois, afinal, gostaríamos de separar os dados no espaço original, não é mesmo?

Na verdade, nosso objetivo é que os dados sejam separados, **não importa em que espaço**! Se conseguirmos encontrar um espaço onde há separabilidade através da aplicação de um feature map, bastaria **aplicar o mesmo feature map** aos dados de treino e de teste, e a separabilidade sempre estará garantida!

Muito legal, não é mesmo?

E claro, embora tenhamos visto um feature map do tipo $\Phi : \mathbb{R}^1 \rightarrow \mathbb{R}^2$, eles podem assumir as mais diferentes formas! Em geral, podemos definir um feature map genérico como $\Phi : \mathcal{X} \rightarrow \mathcal{Z}$ (de forma concreta, $\Phi : \mathbb{R}^m \rightarrow \mathbb{R}^n$, para $m, n$ dimensões arbitrárias!)

Mais um exemplo, $\Phi : \mathbb{R}^2 \rightarrow \mathbb{R}^3$:

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/105e0718-f938-4d09-919a-b37f79b410f7.PNG width=400>

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/526d476e-614b-4acf-b6c5-41cad5567b98.gif width=400>

Finalmente, agora temos todos os elementos necessários para entender o que são as SVMs:

> Uma **Support Vector Machine** nada mais é que **um classificador de margem suave** treinado **no espaço de features**. Portanto, este classificador pressupõe a aplicação prévia de um **feature map** aos dados no espaço de input.

<div class="warning" style='padding:0.1em; background-color:#E9D8FD; color:#69337A'>
<span>
<h2>Conclusões</h2>
<ul>
<li>Um classificador de margem máxima tem alto potencial de generalização, para dados linearmente separáveis;<br><br>
<li>Um classificador de margem suave (vetores de suporte) introduz um pouco mais de viés ao permitir erros de treino, mas, com isso, aumenta o potencial de generalização, e pode ser aplicável a dados que não são exatamente linearmente separáveis;<br><br>
<li>A fronteira de decisão de um classificador de margem sempre será linear no espaço de features;<br><br>
<li>Se os dados não forem linearmente separáveis no espaço original (de input), podemos fazer um feature map para levá-los para um espaço de features onde haja separabilidade linear;
</ul>
</span>
<br>
</div>

Da discussão acima, ficou claro que é justamente o feature map que dá grande poder às SVMs. De fato, a possibilidade de conseguirmos separabilidade linear é algo formidável!

Neste contexto, uma pergunta natural é: como escolher um bom feature map? Formalmente, existem infinitos feature maps possíveis! Como escolher, dentre infinitas opções, exatamente o mapa exato que nos garante separabilidade linear no espaço de features? Embora esta pergunta não seja fácil de responder, existem algumas técnicas para nos ajudar a escolher bons feature maps (vamos discutir sobre isso mais a diante).

Além disso, existe um segundo problema, ainda maior: suponha que queiramos introduzir um kernel que leva os pontos para um espaço de features de altíssima dimensionalidade (algo como $\Phi : \mathbb{R}^2 \mapsto \mathbb{R}^{10000}$).

É de se esperar que este seja um feature map **operacionalmente custoso** de ser calculado, não é mesmo? Imagina, ter que aplicar esta transformação para todos os pontos de treino, e depois de teste! De fato, isso rapidamente se torna computacionalmente impraticável...

Pra solucionar este problema, foi introduzido o uso de **funções de kernel** para capturar um aspecto importante dos feature maps! 

Mas, pra entendermos a importância das funções de kernel, é necessário entendermos uma coisa:

> A hipótese do SVM depende apenas do **produto interno** entre as observações no espaço de features!!

A hipótese é a seguinte:

$$f_{H, \alpha}(\vec{x}) = \text{sign} \left ( \sum_{i=1}^N y_i \alpha_i \langle \Phi(\vec{x}) , \Phi(\vec{x}_i) \rangle + b\right ) $$

Entendo os termos:

- Cada $\vec{x}_i$, $i = 1, 2, \cdots, N$ é uma das $N$ **observações de treino**; e cada $y_i$ é o respectivo target;
- Cada $\alpha_i$ é um [multiplicador de lagrange](https://en.wikipedia.org/wiki/Lagrange_multiplier). Podemos entendê-los simplesmente como **coeficientes numéricos associados a cada observação de treino**, sendo que $\alpha_i > 0$. Podemos entender estes coeficientes como sendo substitutos ao $\vec{w}$ (uma representação [dual](https://en.wikipedia.org/wiki/Duality_(optimization)) do hiperplano);
- $\langle \Phi(\vec{x}) , \Phi(\vec{x}_i) \rangle$ é o produto interno entre a observação de teste $\vec{x}$ e cada observação de treino $\vec{x}_i$, **no espaço de features**, ou seja, após a aplicação do feature map!

> **Obs.:** O produto interno (na forma de produto escalar) é dado por: $\langle \vec{a} , \vec{b} \rangle \equiv \vec{a} \cdot \vec{b} = a_1 b_1 + a_2 b_2 + \cdots + a_d b_d = \sum_{i=1}^d a_i b_i$


Note que, de fato, apenas o produto interno aparece! E é isso que permite o uso das funções de kernel.

<div class="warning" style='padding:0.1em; background-color:#E9D8FD; color:#69337A'>
<span>
<h2>Conclusões</h2>
<ul>
<li>Um classificador de margem máxima tem alto potencial de generalização, para dados linearmente separáveis;<br><br>
<li>Um classificador de margem suave (vetores de suporte) introduz um pouco mais de viés ao permitir erros de treino, mas, com isso, aumenta o potencial de generalização, e pode ser aplicável a dados que não são exatamente linearmente separáveis;<br><br>
<li>A fronteira de decisão de um classificador de margem sempre será linear no espaço de features;<br><br>
<li>Se os dados não forem linearmente separáveis no espaço original (de input), podemos fazer um feature map para levá-los para um espaço de features onde haja separabilidade linear;<br><br>
<li>A única coisa que importa pro SVM é o produto interno entre as observações no espaço de features;
</ul>
</span>
<br>
</div>

____
____
____

## 4) Funções de kernel

Uma função de kernel $\kappa$ nada mais é que uma **medida de similaridade** entre dois vetores $\vec{x}$ e $\vec{x}'$ (que no nosso caso, são observações). Definimos como:

$$\boxed{\begin{align*}
\kappa \ \colon \ & \mathcal{X} \times \mathcal{X} \longrightarrow \mathbb{R} \\
& (\vec{x}, \vec{x}') \longmapsto \kappa(\vec{x}, \vec{x}')
\end{align*}}$$

Ou seja, um kernel é uma função que, dadas duas observações $\vec{x}$ e $\vec{x}'$, retorna um número real que caracteriza o quão similar as duas observações são entre si.

**Ponto fundamental**: uma função de kernel permite que **o produto escalar** entre duas observações seja calculado **no espaço de features**, sem que precisemos **explicitamente levar as observações pro espaço de features**.

Ou seja, conseguimos **evitar** que o feautre map, que costuma ser custoso computacionalmente, seja explicitamente aplicado!

E uma vez que pro SVM apenas o produto interno interessa, podemos usar diretamente a função de kernel, que é muito mais computacionalmente simples que o feature map explícito!

Esse é o chamado **kernel trick**.

### Importantíssimo:

Assim, definimos **o produto interno no espaço de features** como sendo as medidas de similaridade entre os pontos neste espaço:

$$ \kappa_\Phi(\vec{x}, \vec{x}') = \langle \Phi(\vec{x}) , \Phi(\vec{x}') \rangle \ ,$$

onde o subscrito $\Phi$ deixa explícito que a dependência funcional explícita da função de kernel, depende da escolha do feature map considerado!

Agora o ponto muitíssimo importante:

> Note que $\kappa_\Phi$ nos dá **uma forma de calcular $\langle \Phi(\vec{x}) , \Phi(\vec{x}') \rangle$ diretamente a partir de $\vec{x}$ e $\vec{x}'$!**
<br><br>
Ou seja, apesar de **considerarmos implicitamente o feature map $\Phi$ na dependêcia funcional de $\kappa_\Phi$**, nós não precisamos efetivamente calcular $\Phi(\vec{x})$ e $\Phi(\vec{x}')$!
<br><br>
Com isso, reduzimos drasticamente os custos computacionais associados à hipótese!

_____________

#### Exemplo de aplicação do kernel trick

Considere que temos $\mathcal{X} = \mathbb{R}^2$, isto é, $\vec{x} = (X_1, X_2)$, um espaço de input de duas dimensões.

Vamos considerar um feature map $\Phi : \mathbb{R}^2 \rightarrow \mathbb{R}^6$, ou seja, teremos $\mathcal{Z} = \mathbb{R}^6$ como espaço de features. Explicitamente, a aplicação do feature map é:

$$\Phi(\vec{x}) = \left(1, X_1^2, X_2^2, \sqrt{2}X_1, \sqrt{2}X_2, \sqrt{2}X_1 X_2 \right )$$

Assim, tomando duas observações genéricas $\vec{x}$ e  $\vec{x}'$, temos:

$$\begin{align*}
\langle \Phi(\vec{x}) , \Phi(\vec{x}') \rangle &= \langle \left(1, X_1^2, X_2^2, \sqrt{2}X_1, \sqrt{2}X_2, \sqrt{2}X_1 X_2 \right ), \left(1, X'^2_1, X'^2_2, \sqrt{2}X'_1, \sqrt{2}X'_2, \sqrt{2}X'_1 X'_2 \right )\rangle =
\\
\\
&= 1 + X_1^2 X'^2_1 + X_2^2 X'^2_2 + \left ( \sqrt{2}X_1 \right) \left ( \sqrt{2}X'_1 \right) + \left ( \sqrt{2}X_2 \right )\left ( \sqrt{2}X'_2  \right )+ \left (\sqrt{2}X_1 X_2 \right )\left (\sqrt{2}X'_1 X'_2 \right )  =
\\
\\
&= 1 + X_1^2 X'^2_1 + X_2^2 X'^2_2 + 2 \left ( X_1 X'_1 + X_2 X'_2 + X_1 X'_1 X_2 X'_2 \right ) 
\end{align*}$$

Vamos rearranjar os termos acima da seguinte maneira:

$$\begin{align*}
\langle \Phi(\vec{x}) , \Phi(\vec{x}') \rangle &= 1 + \left ( X_1^2 X'^2_1 + X_2^2 X'^2_2 + 2  X_1 X'_1 X_2 X'_2  \right ) + 2 \left ( X_1 X'_1 + X_2 X'_2\right ) =
\\
\\
&= 1 + \left ( X_1 X'_1 + X_2 X'_2\right )^2 + 2 \left ( X_1 X'_1 + X_2 X'_2\right ) = 
\\ 
\\
&= \Big ( 1 + \left ( X_1 X'_1 + X_2 X'_2 \right ) \Big)^2
\end{align*}$$

Agora, note que: $\langle \vec{x} , \vec{x}' \rangle = \langle (X_1, X_2), (X'_1, X'_2) \rangle = X_1 X'_1 + X_2 X'_2$, exatamente como aparece no resultado acima! Sendo assim, temos: 

$$\langle \Phi(\vec{x}) , \Phi(\vec{x}') \rangle = \left ( 1 + \langle \vec{x} , \vec{x}' \rangle \right)^2$$

Ou seja, mostramos que é possível escrever o produto interno entre os vetores **no espaço de features** em termos (do produto interno) dos vetores **no espaço de input!**. Oras, isso é justamente o kernel:

$$ \kappa_\Phi(\vec{x}, \vec{x}') = \left ( 1 + \langle \vec{x} , \vec{x}' \rangle \right)^2 = \langle \Phi(\vec{x}) , \Phi(\vec{x}') \rangle$$

Este é o ponto! Fizemos o exemplo a seguir para ver de fato como a utilização do kernel é correspondente ao produto interno das observações no espaço de features! Isto é, de fato, $\kappa_\Phi(\vec{x}, \vec{x}') = \langle \Phi(\vec{x}) , \Phi(\vec{x}') \rangle$!

É por isso que dizemos que o kernel nos permite **calcular o produto interno entre dois vetores no espaço de features** sem explicitamente "**ter que visitar**" o espaço de features! Este é o ganho de eficiência que os kernels proporcionam!
_____________

Assim, definimos **o produto interno no espaço de features** como sendo as medidas de similaridade entre os pontos neste espaço:

$$ \kappa_\Phi(\vec{x}, \vec{x}') = \langle \Phi(\vec{x}) , \Phi(\vec{x}') \rangle \ ,$$

E a hipótese é reescrita como:

$$\boxed{f_{H, \alpha}(\vec{x}) = \text{sign} \left ( \sum_{i=1}^N y_i \alpha_i\kappa_\Phi(\vec{x}, \vec{x}_i) + b\right )} $$


<div class="warning" style='padding:0.1em; background-color:#E9D8FD; color:#69337A'>
<span>
<h2>Conclusões</h2>
<ul>
<li>Um classificador de margem máxima tem alto potencial de generalização, para dados linearmente separáveis;<br><br>
<li>Um classificador de margem suave (vetores de suporte) introduz um pouco mais de viés ao permitir erros de treino, mas, com isso, aumenta o potencial de generalização, e pode ser aplicável a dados que não são exatamente linearmente separáveis;<br><br>
<li>A fronteira de decisão de um classificador de margem sempre será linear no espaço de features;<br><br>
<li>Se os dados não forem linearmente separáveis no espaço original (de input), podemos fazer um feature map para levá-los para um espaço de features onde haja separabilidade linear;<br><br>
<li>A única coisa que importa pro SVM é o produto interno entre as observações no espaço de features;<br><br>
<li>Uma função de kernel permite que calculemos o produto interno no espaço de features sem a necessidade de explicitamente aplicar o feature map.
</ul>
</span>
<br>
</div>

Da mesma forma que existem infinitos feature maps possíveis, a variedade de kernels também é imensa! Apesar dos kernels oferecerem uma vantagem operacional absurda com relação à aplicação explícita do feature map, o problema de escolha de um kernel adequado para um determinado problema ainda existe.

Na prática, existem algumas formas de propor kernels, mas este não é um tema fácil. Existe todo um conjunto de métodos e técnicas que se utilizam de kerneles para tarefas de aprendizagem, os chamados [métodos de kernel](https://en.wikipedia.org/wiki/Kernel_method).

Apesar da enorme liberdade no design de kernels, há algumas classes particulares de kernels que são comumente utilizados:

- Linear kernel: $\kappa_\Phi(\vec{x}, \vec{x}') = \langle \vec{x}, \vec{x}'\rangle $
<br><br>
- Polynomial kernel: $\kappa_\Phi(\vec{x}, \vec{x}') = (\gamma \langle \vec{x}, \vec{x}'\rangle + r)^d$
<br><br>
- Radial Basis Function (RBF) kernel: $\kappa_\Phi(\vec{x}, \vec{x}') = \exp(-\gamma \|\vec{x}-\vec{x}'\|^2)$
<br><br>
- Sigmoid kernel: $\kappa_\Phi(\vec{x}, \vec{x}') = \tanh(\gamma \langle \vec{x},\vec{x}'\rangle + r)$

> No exemplo explícito que fizemos acima, usamos justamente um kernel polinomial com $r=\gamma=1$ e $d=2$!

Note que a dependência funcional dos kernels muda, dependendo exatamente do feature map específico que eles representam. No entanto, em todos os casos, as features no espaço de input são utilizadas, o que garante a eficiência!

__________

Agora que entendemos o SVM, vamos ver sua aplicação com o sklearn a um problema de classificação!

____
____
____


## 5) SVM na prática com o sklearn

Para construir um classificador SVM com o sklearn, basta usar a classe [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

Vamos utilizar o dataset German Credit Risk!

> Para a construção de um modelo SVM, é **muito importante que os dados sejam normalizados!**

O motivo é bem simples: como vimos acima, o SVM é completamente baseado no kernel, que por sua vez é dado por um produto interno. Já o produto interno, é altamente dependente da **escala das features** (lembre-se, $\langle \Phi(\vec{x}) , \Phi(\vec{x}') \rangle = \left | \Phi(\vec{x}) \right | \left | \Phi(\vec{x}') \right | \cos \left ({\theta_{\vec{x}, \vec{x}'}} \right )$, isto é, a norma dos vetores influencia o produto interno!).

Portanto, para evitar que efeitos de escala influenciem a classificação, a normalização é um passo extremamente importante!

In [3]:
df = pd.read_csv("../datasets/german_credit_data.csv", index_col=0)

X = df.select_dtypes(include=np.number)
y = df["Risk"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Agora, vamos instanciar a classe do [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)!

Perceba que há muitos hiperparâmetros. Vale a pena estudar um pouco mais o fundo o funcionamento de cada um, e sua influência. Para algumas dicas práticas do uso de SVMs com o sklearn, [clique aqui!](https://scikit-learn.org/stable/modules/svm.html#tips-on-practical-use)

Na prática, os principais hiperparâmetros serão `C` e o `gamma` (se o kernel escolhido utilizar este parâmetro, como, por exemplo, o kernel rbf: por isso, o hiperparâmetro `kernel` também é importante!):

- `C`: é um parâmetro de regularização, relacionado com a "suavidade" da margem. Ele controla o tradeoff entre a complexidade da fronteira de decisão, e erros de classificação que são permitidos. Quanto **menor** o C, mais suave será a fronteira de decisão, pois mais erros de classificação são permitidos (isto é, a margem fica **mais larga**); quanto **maior** C, a tolerância a erros de classificação é menor (e a margem fica menos suave, mais complexa);

<img src="https://learnopencv.com/wp-content/uploads/2018/07/svm-parameter-c-example.png" width=500>

- `gamma`: define a influência que cada ponto tem na fronteira de decisão. É a "abertura" do kernel: quanto **maior** o gamma, a influência é de mais curto alcance, e vice-versa;

<img src="https://sgao323.gitbooks.io/artificial-intelligence-projects/content/assets/svm_gamma.png" width=400>

- `kernel`: as opções disponíveis são `linear`, `poly`, `rbf` e `sigmoid`, que apresentamos acima. Também é possível utilizar um kernel personalizado pré-calculado (neste caso, usamos a opção `precomputed`).


No que diz respeito a `C` e `gamma`, é importante que valores adequados sejam encontrados com o uso de Grid/Random search, usando **valores exponencialmente espaçados**.

In [4]:
from sklearn.svm import SVC

In [5]:
# probability=True pra que possamos calcular o predict_proba, que é usado na função de métricas!

pipe_svc = Pipeline([("standard_scaler", StandardScaler()),
                     ("svc", SVC(probability=True, random_state=42))])

pipe_svc.fit(X_train, y_train)

_ = clf_metrics_train_test(pipe_svc, X_train, y_train, X_test, y_test, cutoff=0.5, 
                           print_plot=True, plot_conf_matrix=False, print_cr=True, pos_label="bad")

Métricas de avaliação de treino - com cutoff = 0.50
              precision    recall  f1-score   support

         bad       0.75      0.22      0.34       240
        good       0.74      0.97      0.84       560

    accuracy                           0.74       800
   macro avg       0.75      0.59      0.59       800
weighted avg       0.75      0.74      0.69       800


################################################################################

Métricas de avaliação de teste - com cutoff = 0.50
              precision    recall  f1-score   support

         bad       0.44      0.13      0.21        60
        good       0.71      0.93      0.81       140

    accuracy                           0.69       200
   macro avg       0.58      0.53      0.51       200
weighted avg       0.63      0.69      0.63       200



Um hiperparâmetro importante, sobretudo quando temos classes desbalanceadas, é o `class_weight="balanced"`.

O que isso faz é atribuir um **peso maior na função de custo** a erros cometidos em observações da classe **menos frequente** na base, e vice-versa. 

Isso corresponde a implicitamente "focar mais" nas observações menos frequentes, com o objetivo de compensar o desbalanceamento das classes!

In [6]:
pipe_svc = Pipeline([("standard_scaler", StandardScaler()),
                     ("svc", SVC(class_weight="balanced", probability=True, random_state=42))])

pipe_svc.fit(X_train, y_train)

_ = clf_metrics_train_test(pipe_svc, X_train, y_train, X_test, y_test, cutoff=0.5, 
                           print_plot=True, plot_conf_matrix=False, print_cr=True, pos_label="bad")

Métricas de avaliação de treino - com cutoff = 0.50
              precision    recall  f1-score   support

         bad       0.76      0.28      0.41       240
        good       0.76      0.96      0.85       560

    accuracy                           0.76       800
   macro avg       0.76      0.62      0.63       800
weighted avg       0.76      0.76      0.72       800


################################################################################

Métricas de avaliação de teste - com cutoff = 0.50
              precision    recall  f1-score   support

         bad       0.50      0.18      0.27        60
        good       0.72      0.92      0.81       140

    accuracy                           0.70       200
   macro avg       0.61      0.55      0.54       200
weighted avg       0.66      0.70      0.65       200



Agora, podemos montar um gridsearch para otimizar o modelo:

**Obs importante: cuidado com o `max_iter`!** É prudente limitá-lo para evitar que o SVM fique em loop infinito sem convergir!

In [7]:
np.logspace(-3, 3, 10)

array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03])

In [8]:
pipe_svc = Pipeline([("ss", StandardScaler()),
                     ("svc", SVC(max_iter=10000, probability=True, random_state=42))])

param_grid_svc = {"svc__kernel" : ["rbf", "poly", "sigmoid"],
                  "svc__C" : np.logspace(-3, 3, 10)}

splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_svc = GridSearchCV(estimator=pipe_svc,
                       param_grid=param_grid_svc,
                       scoring="f1_weighted",
                       cv=splitter,
                       verbose=10,
                       n_jobs=-1,
                       return_train_score=True)

grid_svc.fit(X_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('ss', StandardScaler()),
                                       ('svc',
                                        SVC(max_iter=10000, probability=True,
                                            random_state=42))]),
             n_jobs=-1,
             param_grid={'svc__C': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                         'svc__kernel': ['rbf', 'poly', 'sigmoid']},
             return_train_score=True, scoring='f1_weighted', verbose=10)

In [9]:
grid_svc.best_params_

{'svc__C': 10.0, 'svc__kernel': 'rbf'}

In [10]:
_ = clf_metrics_train_test(grid_svc, X_train, y_train, X_test, y_test, cutoff=0.5, 
                           print_plot=True, plot_conf_matrix=False, print_cr=True, pos_label="bad")

Métricas de avaliação de treino - com cutoff = 0.50
              precision    recall  f1-score   support

         bad       0.85      0.25      0.39       240
        good       0.75      0.98      0.85       560

    accuracy                           0.76       800
   macro avg       0.80      0.62      0.62       800
weighted avg       0.78      0.76      0.71       800


################################################################################

Métricas de avaliação de teste - com cutoff = 0.50
              precision    recall  f1-score   support

         bad       0.69      0.18      0.29        60
        good       0.73      0.96      0.83       140

    accuracy                           0.73       200
   macro avg       0.71      0.57      0.56       200
weighted avg       0.72      0.73      0.67       200



E esse é o SVM para classificação! :)

O SVM també pode ser utilizado para regressão, vamos ver a seguir!

In [11]:
# lição de casa: continuar o grid search, ver se consegue mais que 0.67 no f1 ponderado de teste!

____
____
____

## 6) SVM para regressão

Embora fizemos nossa apresentação do SVM como um classificador, também é possível utilizar este método para regressão!

Todos os elementos do classificador SVM (margem, kernel, etc.) também são relevantes aqui.

A ideia é bem simples: utilização de um kernel para que um modelo de **regressão linear seja treinado no espaço de features**. No espaço de inputs, este modelo é refletido como uma regressão não-linear (da mesma forma que, no caso de classificação, fronteiras de decisão lineares no espaço de features são refletidas como fronteiras não-lineares no espaço de input).

A principal diferença é que o conceito de margem também está presente, de modo que **apenas alguns pontos efetivamente vão contriuir para a regressão**. Neste caso, são os pontos **dentro da margem** (região conhecida como $\epsilon-$tubo) que serão estes vetores de suporte. Ou seja, pontos que estão fora da margem não contribuem para a função de custo.

<img src=https://www.saedsayad.com/images/SVR_5.png width=600>

**Hiperparâmetro `epsilon`**

Este hiperparâmetro determina o epsilon-tubo, dentro do qual nenhuma penalidade é aplicada aos pontos.

Se recomenda testar valores entre 1e-3 e 1 ([veja aqui](http://adrem.uantwerpen.be/bibrem/pubs/IJCNN2007.pdf) maiores detalhes!)

Uma comparação entre classificadores e regressores SVM:

<img src=https://miro.medium.com/max/1100/1*XE9jt0r1yAW8LnliQ3mllQ.png width=600>

A classe de regressores SVM no sklearn é a [SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html). Vamos vê-la em ação!

In [12]:
df = pd.read_csv("../datasets/house_prices.csv")

X = df.drop(columns=["Id", "SalePrice"])
y = df["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.select_dtypes(include=np.number).dropna(axis="columns")
X_test = X_test.loc[:, X_train.columns]

In [13]:
y_train

254     145000
1066    178000
638      85000
799     175000
380     127000
         ...  
1095    176432
1130    135000
1294    115000
860     189950
1126    174000
Name: SalePrice, Length: 1168, dtype: int64

In [14]:
from sklearn.svm import SVR

In [17]:
pipe_svr = Pipeline([("ss", StandardScaler()),
                     ("svr", SVR())])

pipe_svr.fit(X_train, y_train)

_ = reg_metrics_train_test(pipe_svr, X_train, y_train, X_test, y_test, 
                           plot=False, dist_resids=False, print_stuff=True)


Métricas de avaliação (dados de treino):

R^2: -0.04
MAE: 54498.41
RMSE: 78919.65
MAPE: 31.11%

################################################################################

Métricas de avaliação (dados de teste):

R^2: -0.02
MAE: 59512.30
RMSE: 88624.65
MAPE: 35.93%


O modelo tá super underfitado!

Vamos ver se conseguimos melhorar com uma busca aleatória, inicialmente

In [19]:
# vamos considerar pro hipeparametro "C" valores uniformemente distribuidos entre 0.0001 e 10000
scipy.stats.uniform(1e-4, 1e4).support()

(0.0001, 10000.0001)

In [20]:
scipy.stats.uniform(1e-3, 1).support()

(0.001, 1.001)

In [21]:
pipe_svr = Pipeline([("ss", StandardScaler()),
                     ("svr", SVR(max_iter=10000))])

param_distros_svr = {"svr__kernel" : ["rbf", "poly", "sigmoid"],
                     "svr__C" : scipy.stats.uniform(1e-4, 1e4),
                     "svr__epsilon" : scipy.stats.uniform(1e-3, 1)}

splitter = KFold(n_splits=5, shuffle=True, random_state=42)

grid_random_svr = RandomizedSearchCV(pipe_svr,
                                    param_distros_svr,
                                    n_iter=100, # aqui que escolhemos o número de combinações amostradas!
                                    cv=splitter,
                                    scoring="neg_mean_absolute_error",
                                    verbose=10,
                                    n_jobs=-1,
                                    return_train_score=True,
                                    random_state=42)

grid_random_svr.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=Pipeline(steps=[('ss', StandardScaler()),
                                             ('svr', SVR(max_iter=10000))]),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'svr__C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000021984C29700>,
                                        'svr__epsilon': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000021984C46A00>,
                                        'svr__kernel': ['rbf', 'poly',
                                                        'sigmoid']},
                   random_state=42, return_train_score=True,
                   scoring='neg_mean_absolute_error', verbose=10)

In [22]:
grid_random_svr.best_params_

{'svr__C': 3109.823317156622,
 'svr__epsilon': 0.32618332202674705,
 'svr__kernel': 'sigmoid'}

In [23]:
_ = reg_metrics_train_test(grid_random_svr, X_train, y_train, X_test, y_test, 
                           plot=False, dist_resids=False, print_stuff=True)


Métricas de avaliação (dados de treino):

R^2: 0.75
MAE: 22006.34
RMSE: 38759.14
MAPE: 11.47%

################################################################################

Métricas de avaliação (dados de teste):

R^2: 0.72
MAE: 23130.68
RMSE: 45932.14
MAPE: 11.92%


Já melhorou bastante!

In [ ]:
# pra casa: continue o tuning de hiperparametros!